In [ ]:
# | default_exp tools.execute_bash

In [ ]:
# | export
from agentic.tools.base import Tool
from agentic.schemas import ExecuteBashParams
import subprocess
from typing import Dict, Any

class ExecuteBashTool(Tool):
    def get_tool_schema(self) -> Dict[str, Any]:
        """Return the OpenAI-compatible schema for execute_bash."""
        return {
                "type": "function",
                "function": {
                    "name": "execute_bash",
                    "description": "bash execution with working directory control, environment variables, timeout management, and detailed output formatting.",
                    "parameters": {
                        "type": "object",
                        "properties": {
                            "command": {"type": "string", "description": "Bash command to execute"},
                            "working_dir": {"type": "string", "description": "Working directory for command execution"},
                            "timeout": {"type": "integer", "description": "Timeout in seconds (default: 30)"},
                            "env_vars": {"type": "object", "description": "Additional environment variables"},
                            "capture_output": {"type": "boolean", "description": "Capture stdout/stderr (default: true)"},
                            "shell": {"type": "string", "description": "Shell to use (default: /bin/bash)"},
                            "summary": {"type": "string", "description": "Brief description of what the command does"}
                        },
                        "required": ["command"]
                    }
                }
            }

    def execute(self, **kwargs) -> Dict[str, Any]:
        """Execute a bash command with specified parameters."""
        try:
            params = ExecuteBashParams(**kwargs)
            shell = params.shell or "/bin/bash"
            
            try:
                result = subprocess.run(
                    params.command,
                    shell=True,
                    capture_output=params.capture_output,
                    text=True,
                    timeout=params.timeout,
                    cwd=params.working_dir,
                    env=params.env_vars
                )
                
                return {
                    "success": True,
                    "stdout": result.stdout,
                    "stderr": result.stderr,
                    "exit_status": result.returncode,
                    "summary": params.summary or f"Executed command: {params.command}"
                }
                
            except subprocess.TimeoutExpired:
                return {"error": f"Command timed out after {params.timeout} seconds"}
            except subprocess.SubprocessError as e:
                return {"error": f"Command execution failed: {str(e)}"}
                
        except Exception as e:
            return {"error": f"Invalid parameters: {str(e)}"}